In [1]:
import pandas as pd
from backtesting import Backtest
from library import Client, Payload, ConcreateStrategy
from datetime import datetime
import numpy as np
pd.options.display.float_format = "{:6.2f}".format

client = Client()

/usr/local/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

{'in_sample_start': datetime.datetime(2010, 1, 1, 0, 0), 'in_sample_end': datetime.datetime(2011, 12, 31, 0, 0), 'out_sample_start': datetime.datetime(2012, 1, 1, 0, 0), 'out_sample_end': datetime.datetime(2012, 12, 31, 0, 0)}
{'in_sample_start': datetime.datetime(2013, 1, 1, 0, 0), 'in_sample_end': datetime.datetime(2014, 12, 31, 0, 0), 'out_sample_start': datetime.datetime(2015, 1, 1, 0, 0), 'out_sample_end': datetime.datetime(2015, 12, 31, 0, 0)}


In [31]:
class My_Strategy(ConcreateStrategy):
    def init(self):
        self.use_atr()
        self.use_donchian_channel()
        self.use_half_donchian_channel()

    def next(self):
        # DCによる手仕舞い
        if self.is_half_donchian_channel_updated_lowest() and self.position.is_long:
            self.position.close()
            return

        # 買い注文
        if (
            self.is_donchian_channel_updated_highest()
            and not self.position.is_long
        ):
            self.buy_with_atr()
            return

In [46]:
bt = Backtest(
    client.get_in_cache(Payload(
        tickers="spy",
        start=datetime.strptime("1990-01-01", "%Y-%m-%d"),
        end=datetime.strptime("2023-01-01", "%Y-%m-%d"))
    ),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)

# optimize = bt.optimize(
#     maximize="SQN",
#     prop_donchian_channel_period = range(10, 200, 10),
#     prop_atr_stop_loss = 10, # 実際反応しないような値を入れておく
#     prop_atr_take_profit = np.arange(1.8, 8.0, 0.1).tolist(),
# )

# SQN:2.7 My_Strategy(prop_donchian_channel_period=120,prop_atr_stop_loss=10,prop_atr_take_profit=5.600000000000003)
optimize = bt.optimize(
    maximize="SQN",
    prop_donchian_channel_period = 120,
    prop_atr_stop_loss = 10, # 実際反応しないような値を入れておく
    prop_atr_take_profit = 5.6
)

# 出力
bt.plot()
print(optimize)
print(optimize._strategy)

/usr/local/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1178 configurations.
  output = _optimize_grid()


Start                     1993-01-29 00:00:00
End                       2022-12-30 00:00:00
Duration                  10927 days 00:00:00
Exposure Time [%]                       55.77
Equity Final [$]                  33769044.90
Equity Peak [$]                   36205692.94
Return [%]                             237.69
Buy & Hold Return [%]                  770.40
Return (Ann.) [%]                        4.15
Volatility (Ann.) [%]                   10.46
Sharpe Ratio                             0.40
Sortino Ratio                            0.57
Calmar Ratio                             0.21
Max. Drawdown [%]                      -19.71
Avg. Drawdown [%]                       -2.05
Max. Drawdown Duration     1655 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                                   78
Win Rate [%]                            64.10
Best Trade [%]                          13.53
Worst Trade [%]                         -8.93
Avg. Trade [%]                    